In [508]:
import polars as pl
import numpy as np

In [509]:
#lista trimestral a remplazar ene_march, april_june, 
#july_september, october_december
trim = "ene_march"
trim2 = "april_june"
trim3 = "july_september"
trim4 = "october_december"

In [510]:
def ajustar_rangos(df, columna_distancia, lim_superior, lim_inferior, step, columna_fecha):
    """
    df: DataFrame de Polars
    columna_distancia: Nombre de la columna de distancia a filtrar
    lim_superior: Valor superior del rango de distancia
    lim_inferior: Valor inferior del rango de distancia
    step: Tamaño del paso para la distancia
    columna_fecha: Nombre de la columna de fecha para agrupar por días
    """
    total_filas = df.shape[0]  # Número total de filas en el DataFrame
    resultado = []  # Lista para almacenar las filas filtradas y ajustadas
    
    # Extraer solo la fecha de tpep_pickup_datetime (sin hora)
    df = df.with_columns(pl.col(columna_fecha).dt.date().alias('grupo_fecha'))

    # Iterar sobre cada grupo de fechas
    for fecha_grupo, df_grupo in df.group_by('grupo_fecha'):  # Cambiar a group_by
        # Iterar sobre los límites del rango de distancia
        for limite in np.arange(lim_superior, lim_inferior, -step):
            rango_superior = limite
            rango_inferior = limite - step
            
            # Filtrar el DataFrame del grupo de fecha por el rango de distancia
            df_rango = df_grupo.filter(
                (pl.col(columna_distancia) > rango_inferior) & (pl.col(columna_distancia) <= rango_superior)
            )
            
            # Calcular la cantidad a mantener en este rango de distancia y grupo de fecha
            cantidad = int(df_rango.shape[0] / total_filas * 100 * 1000)
            
            # Si hay más filas de las necesarias, se reducen a la cantidad especificada
            if df_rango.shape[0] > cantidad:
                df_rango = df_rango.head(cantidad)  # O usa .sample(n=cantidad) para aleatoriedad
            
            # Agregar el DataFrame ajustado a la lista de resultados
            resultado.append(df_rango)

    # Concatenar todos los DataFrames ajustados en uno solo
    df_final = pl.concat(resultado)
    
    return df_final

In [511]:
def escalado(serie):
    # Calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    
    # Calcular el rango intercuartílico (IQR)
    iqr = q3 - q1
    
    # Aplicar el factor de 1.5 para calcular los límites
    iqr_1 = q1 - (1.5 * iqr)
    iqr_3 = q3 + (1.5 * iqr)
    
    return iqr_1, iqr_3

In [512]:
def limpiar_nombres_columnas(df):
    df.columns = [col.strip().lower() for col in df.columns]
    return df

In [513]:
part_ruta = "data_temporal_green/2023_particion_green_"

In [514]:
data_ref_schema = pl.read_parquet("data_trimestral/2014_green_taxis.parquet")

In [515]:
data_ref_schema_two = pl.read_parquet('data_temporal_green/2016_particion_green_ene_march.parquet')

In [516]:
data_ref_schema = limpiar_nombres_columnas(data_ref_schema)
data_ref_schema_two = limpiar_nombres_columnas(data_ref_schema_two)

In [517]:
df_one = pl.read_parquet(f"{part_ruta}{trim}.parquet")

In [518]:
df_two = pl.read_parquet(f"{part_ruta}{trim2}.parquet")

In [519]:
df_three = pl.read_parquet(f"{part_ruta}{trim3}.parquet")

In [520]:
df_four = pl.read_parquet(f"{part_ruta}{trim4}.parquet")

In [521]:
df_one = limpiar_nombres_columnas(df_one)

In [522]:
df_two = limpiar_nombres_columnas(df_two)

In [523]:
df_three = limpiar_nombres_columnas(df_three)

In [524]:
df_four = limpiar_nombres_columnas(df_four)

In [525]:
lim_inferior1, lim_superior1 = escalado(df_one["trip_distance"])

df_one = df_one.filter(
    (df_one['trip_distance'] >= lim_inferior1) & 
    (df_one['trip_distance'] <= lim_superior1)
)

In [526]:
df_one = ajustar_rangos(df_one,"trip_distance",lim_superior1,lim_inferior1,1,'lpep_pickup_datetime')

In [527]:
lim_inferior2, lim_superior2 = escalado(df_two["trip_distance"])

df_two = df_two.filter(
    (df_two['trip_distance'] >= lim_inferior2) & 
    (df_two['trip_distance'] <= lim_superior2)
)

In [528]:
df_two = ajustar_rangos(df_two,"trip_distance",lim_superior2,lim_inferior2,1,'lpep_pickup_datetime')

In [529]:
lim_inferior3, lim_superior3 = escalado(df_three["trip_distance"])

df_three = df_three.filter(
    (df_three['trip_distance'] >= lim_inferior3) & 
    (df_three['trip_distance'] <= lim_superior3)
)

In [530]:
df_three = ajustar_rangos(df_three,"trip_distance",lim_superior3,lim_inferior3,1,'lpep_pickup_datetime')

In [531]:
lim_inferior4, lim_superior4 = escalado(df_four["trip_distance"])

df_four = df_four.filter(
    (df_four['trip_distance'] >= lim_inferior4) & 
    (df_four['trip_distance'] <= lim_superior4)
)

In [532]:
df_four = ajustar_rangos(df_four,"trip_distance",lim_superior4,lim_inferior4,1,'lpep_pickup_datetime')

In [533]:
columns = data_ref_schema_two.columns

df_one = df_one.select(columns)
df_two = df_two.select(columns)
df_three = df_three.select(columns)
df_four = df_four.select(columns)

In [534]:
df_final =  pl.concat([df_one,df_two,df_three,df_four])

In [535]:
folder_path = 'data_trimestral'
df_final.write_parquet(f'{folder_path}/2023_green_taxis.parquet')

In [552]:
df_ex = pl.read_parquet('data_trimestral/2023_green_taxis.parquet')

In [537]:
#df_ex = df_ex.drop(["improvement_surcharge","trip_type"])

In [538]:
data_ref_schema = data_ref_schema.drop("grupo_fecha")

In [549]:
#orden_columnas = data_ref_schema.columns

#df_ex = df_ex.select(orden_columnas)

In [551]:
#df_ex.write_parquet(f'{folder_path}/2023_green_taxis.parquet')

In [547]:
#df_ex = df_ex.drop_nulls()